# Notebook 01 — Tratamento e Validação dos Dados

Este notebook realiza a limpeza, validação e preparação dos dados brutos extraídos de um CRM simulado.
O objetivo é garantir qualidade e confiabilidade antes das análises e do dashboard.

## Importação das bibliotecas

In [4]:
import pandas as pd
import sqlite3

## Leitura dos dados brutos(excel)

In [5]:
arquivo = "../dados_raw/base_teste_fato_original.xlsx" 

vendas = pd.read_excel(arquivo, sheet_name="Vendas")
produtos = pd.read_excel(arquivo, sheet_name="Produtos")
clientes = pd.read_excel(arquivo, sheet_name="Clientes")
metas = pd.read_excel(arquivo, sheet_name="Metas")

## Visão inicial das tabelas

In [6]:
vendas.head()

,Data,ID_Produto,ID_Cliente,Canal_Venda,Quantidade,Valor_Unitario,Valor_Total
0,2024-06-22,3,27,Marketplace,4,99.9,399.6
1,2024-05-19,4,43,WhatsApp,3,79.9,239.7
2,2024-03-16,7,12,WhatsApp,1,79.9,79.9
3,2024-02-28,10,20,WhatsApp,5,249.9,1249.5
4,2024-04-05,4,48,WhatsApp,2,199.9,399.8


In [7]:
produtos.head()

,ID_Produto,Nome_Produto,Categoria,Custo_Unitario,Preco_Venda
0,1,Cadeira Office,Cadeiras,150,249.9
1,2,Mesa Escritório,Mesas,400,799.9
2,3,Notebook,Eletrônicos,2800,3999.9
3,4,Teclado,Acessórios,80,149.9
4,5,Mouse,Acessórios,40,99.9


In [8]:
clientes.head()

,ID_Cliente,Nome,Cidade,Estado,Tipo_Cliente
0,1,Cliente 1,Cidade 3,RJ,Pessoa Física
1,2,Cliente 2,Cidade 10,SC,Pessoa Jurídica
2,3,Cliente 3,Cidade 3,SP,Pessoa Física
3,4,Cliente 4,Cidade 4,SC,Pessoa Jurídica
4,5,Cliente 5,Cidade 7,SP,Pessoa Jurídica


In [9]:
metas.head()

,Mes,Meta_Faturamento
0,2024-01,59725
1,2024-02,55353
2,2024-03,38115
3,2024-04,35387
4,2024-05,53919


## Verificação de valores null

In [10]:
vendas.isna().sum()

Data              0
ID_Produto        0
ID_Cliente        0
Canal_Venda       0
Quantidade        0
Valor_Unitario    0
Valor_Total       0
dtype: int64

In [11]:
produtos.isna().sum()

ID_Produto        0
Nome_Produto      0
Categoria         0
Custo_Unitario    0
Preco_Venda       0
dtype: int64

In [12]:
clientes.isna().sum()

ID_Cliente      0
Nome            0
Cidade          0
Estado          0
Tipo_Cliente    0
dtype: int64

In [13]:
metas.isna().sum()

Mes                 0
Meta_Faturamento    0
dtype: int64

## Validaão de chaves
Verificar se todos os produtos das vendas existem na base de produtos e se todos os clientes das vendas existem na base de clientes

In [14]:
vendas['ID_Produto'].isin(produtos['ID_Produto']).value_counts()

ID_Produto
True    200
Name: count, dtype: int64

In [15]:
vendas['ID_Cliente'].isin(clientes['ID_Cliente']).value_counts()

ID_Cliente
True    200
Name: count, dtype: int64

In [ ]:
## Analise de distruibuições importantes

In [16]:
vendas['Canal_Venda'].value_counts()

Canal_Venda
WhatsApp       70
Marketplace    67
Site           63
Name: count, dtype: int64

In [17]:
clientes['Tipo_Cliente'].value_counts()

Tipo_Cliente
Pessoa Física      27
Pessoa Jurídica    23
Name: count, dtype: int64

## Ajuste do valor dos produtos
- O preço da venda deve ser o Preco_Venda da tabela de produtos
- O valor total deve ser Quantidade * Valor_Unitario

In [18]:
vendas = vendas.merge(
    produtos[['ID_Produto', 'Preco_Venda']],
    on='ID_Produto',
    how='left'
)

In [19]:
vendas['Valor_Unitario'] = vendas['Preco_Venda']
vendas['Valor_Total'] = vendas['Quantidade'] * vendas['Valor_Unitario']
vendas = vendas.drop(columns=['Preco_Venda'])

## Tratamento de datas

In [21]:
vendas['Data'] = pd.to_datetime(vendas['Data'], errors='coerce')

In [22]:
vendas['Data'].isna().sum()

0

In [23]:
vendas[vendas['Data'] > pd.Timestamp.today()]

,Data,ID_Produto,ID_Cliente,Canal_Venda,Quantidade,Valor_Unitario,Valor_Total


In [24]:
vendas = vendas.sort_values(by='Data')

In [25]:
metas['Mes'] = pd.to_datetime(metas['Mes'], format='%Y-%m')

## Verificação de duplicidades

In [26]:
vendas.duplicated().sum()

0

## Padronização dos nomes das colunas

In [27]:
vendas.columns = vendas.columns.str.lower()
produtos.columns = produtos.columns.str.lower()
clientes.columns = clientes.columns.str.lower()
metas.columns = metas.columns.str.lower()

## Verificação final dos tipos de dados

In [28]:
vendas.dtypes

data              datetime64[ns]
id_produto                 int64
id_cliente                 int64
canal_venda               object
quantidade                 int64
valor_unitario           float64
valor_total              float64
dtype: object

In [29]:
metas.dtypes

mes                 datetime64[ns]
meta_faturamento             int64
dtype: object

## Persistência dos dados tratados em SQLite

Os dados tratados são salvos em um banco SQLite para garantir organização, reprodutibilidade e integração futura com análises e dashboards.

In [30]:
conn = sqlite3.connect('../database/dados_tratados.db')

In [31]:
vendas.to_sql('vendas_tratadas', conn, if_exists='replace', index=False)
produtos.to_sql('produtos_tratados', conn, if_exists='replace', index=False)
clientes.to_sql('clientes_tratados', conn, if_exists='replace', index=False)
metas.to_sql('metas_tratadas', conn, if_exists='replace', index=False)

6

In [32]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)

,name
0,vendas_tratadas
1,produtos_tratados
2,clientes_tratados
3,metas_tratadas


In [33]:
conn.close()

## Conclusões técnicas

Neste notebook, realizei o tratamento e validação dos dados brutos, corrigindo tipos, padronizando datas e garantindo consistência entre as bases. Os dados tratados foram persistidos em SQLite e estão prontos para análise exploratória e visualização.